# Eigenvalue Problems: The Power Method

## Introduction

Eigenvalue problems are fundamental in linear algebra and have widespread applications in physics, engineering, data science, and numerical analysis. Given a square matrix $\mathbf{A} \in \mathbb{R}^{n \times n}$, the eigenvalue problem seeks to find scalar $\lambda$ (eigenvalue) and non-zero vector $\mathbf{v}$ (eigenvector) satisfying:

$$\mathbf{A}\mathbf{v} = \lambda \mathbf{v}$$

## The Power Method

The **Power Method** (also known as power iteration) is an iterative algorithm for finding the **dominant eigenvalue** (the eigenvalue with largest absolute value) and its corresponding eigenvector.

### Algorithm

Given an initial guess $\mathbf{v}_0$, the power method iterates:

$$\mathbf{w}_{k+1} = \mathbf{A}\mathbf{v}_k$$

$$\mathbf{v}_{k+1} = \frac{\mathbf{w}_{k+1}}{\|\mathbf{w}_{k+1}\|}$$

The eigenvalue estimate at iteration $k$ is given by the **Rayleigh quotient**:

$$\lambda_k = \frac{\mathbf{v}_k^T \mathbf{A} \mathbf{v}_k}{\mathbf{v}_k^T \mathbf{v}_k}$$

### Convergence

If the matrix $\mathbf{A}$ has eigenvalues $|\lambda_1| > |\lambda_2| \geq \cdots \geq |\lambda_n|$, the convergence rate is:

$$\text{Error} \sim \mathcal{O}\left(\left|\frac{\lambda_2}{\lambda_1}\right|^k\right)$$

The method converges linearly, with faster convergence when the ratio $|\lambda_2/\lambda_1|$ is small.

### Applications

- **PageRank algorithm**: Finding dominant eigenvector of web graph
- **Principal Component Analysis (PCA)**: Largest eigenvalues of covariance matrix
- **Structural analysis**: Natural frequencies of vibrating systems
- **Quantum mechanics**: Ground state energy calculations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, eig

# Set random seed for reproducibility
np.random.seed(42)

## Implementation of the Power Method

We implement the power method with the following features:
- Normalization at each step to prevent overflow
- Rayleigh quotient for eigenvalue estimation
- Convergence history tracking

In [ ]:
def power_method(A, num_iterations=100, tol=1e-10):
    """
    Power method for finding the dominant eigenvalue and eigenvector.
    
    Parameters:
    -----------
    A : ndarray
        Square matrix
    num_iterations : int
        Maximum number of iterations
    tol : float
        Convergence tolerance
    
    Returns:
    --------
    eigenvalue : float
        Dominant eigenvalue
    eigenvector : ndarray
        Corresponding eigenvector (normalized)
    history : dict
        Convergence history
    """
    n = A.shape[0]
    
    # Initialize with random vector
    v = np.random.rand(n)
    v = v / norm(v)
    
    eigenvalue_history = []
    error_history = []
    
    eigenvalue = 0
    
    for k in range(num_iterations):
        # Matrix-vector multiplication
        w = A @ v
        
        # Normalize
        v_new = w / norm(w)
        
        # Rayleigh quotient for eigenvalue estimate
        eigenvalue_new = (v_new @ A @ v_new) / (v_new @ v_new)
        
        # Track convergence
        eigenvalue_history.append(eigenvalue_new)
        
        # Compute error (change in eigenvalue)
        error = abs(eigenvalue_new - eigenvalue)
        error_history.append(error)
        
        # Check convergence
        if error < tol and k > 0:
            v = v_new
            eigenvalue = eigenvalue_new
            break
        
        v = v_new
        eigenvalue = eigenvalue_new
    
    history = {
        'eigenvalues': eigenvalue_history,
        'errors': error_history,
        'iterations': k + 1
    }
    
    return eigenvalue, v, history

## Example 1: Symmetric Positive Definite Matrix

We first test the power method on a symmetric positive definite matrix, which guarantees real positive eigenvalues.

In [ ]:
# Create a symmetric positive definite matrix
n = 5
B = np.random.rand(n, n)
A1 = B.T @ B + np.eye(n)  # Guarantees positive definiteness

print("Matrix A1:")
print(A1.round(4))
print()

In [ ]:
# Apply power method
eigenvalue_pm, eigenvector_pm, history1 = power_method(A1, num_iterations=50)

# Compare with numpy's eigenvalue decomposition
eigenvalues_np, eigenvectors_np = eig(A1)
idx = np.argmax(np.abs(eigenvalues_np))
dominant_eigenvalue_np = eigenvalues_np[idx].real

print(f"Power Method Result:")
print(f"  Dominant eigenvalue: {eigenvalue_pm:.10f}")
print(f"  Iterations: {history1['iterations']}")
print()
print(f"NumPy Reference:")
print(f"  Dominant eigenvalue: {dominant_eigenvalue_np:.10f}")
print()
print(f"Absolute Error: {abs(eigenvalue_pm - dominant_eigenvalue_np):.2e}")

## Example 2: Convergence Rate Analysis

We analyze how the ratio $|\lambda_2/\lambda_1|$ affects convergence speed by constructing matrices with known eigenvalue structures.

In [ ]:
def create_matrix_with_eigenvalues(eigenvalues):
    """
    Create a matrix with specified eigenvalues using random orthogonal similarity transform.
    """
    n = len(eigenvalues)
    D = np.diag(eigenvalues)
    
    # Random orthogonal matrix via QR decomposition
    Q, _ = np.linalg.qr(np.random.rand(n, n))
    
    # Similar matrix with same eigenvalues
    A = Q @ D @ Q.T
    return A

# Test different eigenvalue ratios
ratios = [0.9, 0.7, 0.5, 0.3, 0.1]
results = []

for ratio in ratios:
    eigenvalues = [10, 10*ratio, 1, 0.5, 0.1]
    A = create_matrix_with_eigenvalues(eigenvalues)
    
    _, _, history = power_method(A, num_iterations=100, tol=1e-12)
    results.append({
        'ratio': ratio,
        'history': history
    })

print("Convergence analysis completed for different λ₂/λ₁ ratios.")

## Visualization

We create comprehensive visualizations showing:
1. Convergence of eigenvalue estimates
2. Error decay on logarithmic scale
3. Comparison of convergence rates

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Eigenvalue convergence for Example 1
ax1 = axes[0, 0]
iterations = range(1, len(history1['eigenvalues']) + 1)
ax1.plot(iterations, history1['eigenvalues'], 'b-o', markersize=4, label='Power Method')
ax1.axhline(y=dominant_eigenvalue_np, color='r', linestyle='--', label='True Value')
ax1.set_xlabel('Iteration', fontsize=11)
ax1.set_ylabel('Eigenvalue Estimate', fontsize=11)
ax1.set_title('Convergence of Eigenvalue Estimate', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Error decay (log scale)
ax2 = axes[0, 1]
errors = history1['errors'][1:]  # Skip first (undefined)
ax2.semilogy(range(2, len(errors) + 2), errors, 'g-o', markersize=4)
ax2.set_xlabel('Iteration', fontsize=11)
ax2.set_ylabel('|λₖ - λₖ₋₁|', fontsize=11)
ax2.set_title('Error Decay (Logarithmic Scale)', fontsize=12)
ax2.grid(True, alpha=0.3)

# Plot 3: Comparison of convergence rates for different ratios
ax3 = axes[1, 0]
colors = plt.cm.viridis(np.linspace(0, 0.9, len(ratios)))

for i, (result, color) in enumerate(zip(results, colors)):
    errors = result['history']['errors'][1:]
    if len(errors) > 1:
        ax3.semilogy(range(2, len(errors) + 2), errors, 
                     color=color, linewidth=2,
                     label=f'λ₂/λ₁ = {result["ratio"]}')

ax3.set_xlabel('Iteration', fontsize=11)
ax3.set_ylabel('Error', fontsize=11)
ax3.set_title('Effect of Eigenvalue Ratio on Convergence', fontsize=12)
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

# Plot 4: Theoretical vs observed convergence rate
ax4 = axes[1, 1]
theoretical_rates = ratios
observed_rates = []

for result in results:
    errors = result['history']['errors']
    if len(errors) > 10:
        # Estimate convergence rate from error decay
        rate = (errors[-1] / errors[10]) ** (1/(len(errors) - 11))
        observed_rates.append(min(rate, 1.0))  # Cap at 1
    else:
        observed_rates.append(0)

ax4.plot(theoretical_rates, theoretical_rates, 'k--', linewidth=2, label='Theoretical')
ax4.scatter(theoretical_rates, observed_rates, s=100, c='red', zorder=5, label='Observed')
ax4.set_xlabel('Theoretical Rate (λ₂/λ₁)', fontsize=11)
ax4.set_ylabel('Observed Convergence Rate', fontsize=11)
ax4.set_title('Theoretical vs Observed Convergence Rate', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to 'plot.png'")

## Example 3: Application to PageRank-like Problem

The power method is famously used in Google's PageRank algorithm. Here we demonstrate a simplified version using a stochastic matrix (rows sum to 1).

In [ ]:
# Create a simple web graph (stochastic matrix)
# 5 pages with links between them
n_pages = 5
G = np.array([
    [0, 1, 1, 0, 0],  # Page 0 links to 1, 2
    [0, 0, 1, 1, 0],  # Page 1 links to 2, 3
    [1, 0, 0, 1, 1],  # Page 2 links to 0, 3, 4
    [0, 1, 0, 0, 1],  # Page 3 links to 1, 4
    [1, 0, 1, 0, 0]   # Page 4 links to 0, 2
], dtype=float)

# Normalize rows to create transition matrix
row_sums = G.sum(axis=1)
P = G / row_sums[:, np.newaxis]

# Add damping factor (standard PageRank)
d = 0.85
M = d * P + (1 - d) / n_pages * np.ones((n_pages, n_pages))

# PageRank uses the transpose (we want column stochastic)
M_T = M.T

print("Transition Matrix (with damping):")
print(M_T.round(4))

In [ ]:
# Apply power method to find PageRank scores
pagerank_eigenvalue, pagerank_vector, _ = power_method(M_T, num_iterations=100)

# Normalize to get probability distribution
pagerank_scores = np.abs(pagerank_vector)
pagerank_scores = pagerank_scores / pagerank_scores.sum()

print("PageRank Scores:")
for i, score in enumerate(pagerank_scores):
    print(f"  Page {i}: {score:.4f}")

print(f"\nDominant eigenvalue: {pagerank_eigenvalue:.6f}")
print("(Should be close to 1 for stochastic matrices)")

## Summary

### Key Takeaways

1. **The Power Method** is a simple iterative algorithm for finding the dominant eigenvalue and eigenvector of a matrix.

2. **Convergence Rate**: The method converges linearly with rate $|\lambda_2/\lambda_1|$. Faster convergence occurs when there is a large gap between the dominant and second eigenvalue.

3. **Rayleigh Quotient**: Provides the eigenvalue estimate $\lambda = \frac{\mathbf{v}^T \mathbf{A} \mathbf{v}}{\mathbf{v}^T \mathbf{v}}$.

4. **Applications**: The method is foundational for PageRank, PCA initialization, and finding dominant modes in physical systems.

### Limitations

- Only finds the **dominant** eigenvalue (modifications like inverse iteration can find others)
- Slow convergence when eigenvalues are closely spaced
- May fail if the dominant eigenvalue is complex or has multiplicity > 1

### Extensions

- **Inverse Power Method**: Finds smallest eigenvalue using $\mathbf{A}^{-1}$
- **Shifted Power Method**: Finds eigenvalue closest to a shift $\sigma$
- **QR Algorithm**: Modern method for computing all eigenvalues